# TEXT2SQL with transformers

In [1]:
import torch
import transformers
print(f"PyTroch Version: {torch.__version__}")
print(f"Transfomers Version: {transformers.__version__}")

PyTroch Version: 1.8.1
Transfomers Version: 4.6.1


## Data to build

In [2]:
from pathlib import Path
import re
import records

schema_re = re.compile(r'\((.+)\)')
num_re = re.compile(r'[-+]?\d*\.\d+|\d+')

db_path = Path("./private")
db = records.Database(f"sqlite:///{db_path / 'samsung_new.db'}")

table_id = "receipts"
table_info = db.query('SELECT sql from sqlite_master WHERE tbl_name = :name', name=table_id).all()[0].sql
schema_str = schema_re.findall(table_info.replace("\n", ""))[0]
schema = {}
for tup in schema_str.split(', '):
    c, t = tup.split()
    schema[c.strip('"')] = t

In [3]:
# sqlite can have NULL, INTEGER, REAL, TEXT, BLOB data type
schema

{'index': 'INTEGER',
 'rcept_no': 'TEXT',
 'reprt_code': 'TEXT',
 'bsns_year': 'INTEGER',
 'corp_code': 'TEXT',
 'stock_code': 'TEXT',
 'fs_div': 'TEXT',
 'fs_nm': 'TEXT',
 'sj_div': 'TEXT',
 'sj_nm': 'TEXT',
 'account_nm': 'TEXT',
 'thstrm_nm': 'TEXT',
 'thstrm_dt': 'TEXT',
 'thstrm_amount': 'INTEGER',
 'frmtrm_nm': 'TEXT',
 'frmtrm_dt': 'TEXT',
 'frmtrm_amount': 'INTEGER',
 'bfefrmtrm_nm': 'TEXT',
 'bfefrmtrm_dt': 'TEXT',
 'bfefrmtrm_amount': 'INTEGER'}

In [4]:
set(schema.values())

{'INTEGER', 'TEXT'}

In [5]:
question = "제 51 기에 삼성전자의 유동자산은 어떻게 돼?"
cls_token = "[CLS]"
table_token = "[T]"
column_token = "[C]"

f"{cls_token} {question} [T] {table_id} " + " ".join([f"{column_token} {col} [{typ}]" for col, typ in schema.items()])

'[CLS] 제 51 기에 삼성전자의 유동자산은 어떻게 돼? [T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [TEXT] [C] bfefrmtrm_dt [TEXT] [C] bfefrmtrm_amount [INTEGER]'

## Build a cumstom Tokenizer

https://huggingface.co/docs/tokenizers/python/latest/pipeline.html

### Normalization

- `normalizers`는 raw text를 더 깨끗하게 만드는 과정이다.
- `NFD` 사용하게 되면 한글은 자음 모음으로 분리된다.
    - NFD(Normalization Form Canonical Decomposition) = 조합형
    - NFC(Normalizaiton Form Canonical Compostion) = 완성형

In [6]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

normalizer = normalizers.Sequence([NFD(), StripAccents()])
# 일반 출력시 합쳐져서 보이지만, for문을 사용하면 분리된다. 
print(normalizer.normalize_str(question))
print("/".join([x for x in normalizer.normalize_str(question)]))

제 51 기에 삼성전자의 유동자산은 어떻게 돼?
ᄌ/ᅦ/ /5/1/ /ᄀ/ᅵ/ᄋ/ᅦ/ /ᄉ/ᅡ/ᆷ/ᄉ/ᅥ/ᆼ/ᄌ/ᅥ/ᆫ/ᄌ/ᅡ/ᄋ/ᅴ/ /ᄋ/ᅲ/ᄃ/ᅩ/ᆼ/ᄌ/ᅡ/ᄉ/ᅡ/ᆫ/ᄋ/ᅳ/ᆫ/ /ᄋ/ᅥ/ᄄ/ᅥ/ᇂ/ᄀ/ᅦ/ /ᄃ/ᅫ/?


### Pre-Tokenization

- `pre_tokenizers`는 텍스트를 더 작은 토큰으로 분리하는 과정이다.
- `Whitespace`는 토큰을 공백을 기준으로 나누면 string의 위치를 같이 반환한다. (start + end) - 기준 regex: `\w+|[^\w\s]+`
- `Punctuation`은 문장 부호를 각각 분리한다. 
- `Digits`를 통해 숫자를 각각 분리할지 말지 결정할 수 있다. 

In [8]:
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Whitespace, Punctuation, Digits

print("Whitespace: ")
print(Whitespace().pre_tokenize_str(question))

print("Digits individual: False")
pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Digits(individual_digits=False)])
print(pre_tokenizer.pre_tokenize_str(question + "????"))

print("Punctuation + Individual Digits")
pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Punctuation(), Digits(individual_digits=True)])
print(pre_tokenizer.pre_tokenize_str(question + "????"))

Whitespace: 
[('제', (0, 1)), ('51', (2, 4)), ('기에', (5, 7)), ('삼성전자의', (8, 13)), ('유동자산은', (14, 19)), ('어떻게', (20, 23)), ('돼', (24, 25)), ('?', (25, 26))]
Digits individual: False
[('제', (0, 1)), ('51', (2, 4)), ('기에', (5, 7)), ('삼성전자의', (8, 13)), ('유동자산은', (14, 19)), ('어떻게', (20, 23)), ('돼', (24, 25)), ('?????', (25, 30))]
Punctuation + Individual Digits
[('제', (0, 1)), ('5', (2, 3)), ('1', (3, 4)), ('기에', (5, 7)), ('삼성전자의', (8, 13)), ('유동자산은', (14, 19)), ('어떻게', (20, 23)), ('돼', (24, 25)), ('?', (25, 26)), ('?', (26, 27)), ('?', (27, 28)), ('?', (28, 29)), ('?', (29, 30))]


### The Model

- `BPE`: Byte-Pair Encoding 토큰화, 자주 등장하는 character를 합쳐서 표현하는 알고리즘
- `Unigram`: 확률적으로 최적의 subword 토큰을 결정
- `WordLevel`: 단어 단위의 토큰화
- `WordPiece`: Google WordPiece 토큰화

In [38]:
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece

### Post-Processing

- `processors` 에서 후처리를 할 수 있다.
- `TemplateProcessing`을 이용해 원하는 형태로 토큰을 분리할 수 있다.

In [42]:
from tokenizers.processors import PostProcessor

In [71]:
from tokenizers.processors import TemplateProcessing

post_processor = TemplateProcessing(
    single="[CLS] $A [T] $B [C] $B",
    pair="[CLS] $A [T] $B:1 [C] $B:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2), ("[T]", 3), ("[C]", 4), ("[INTEGER]", 5), ("[REAL]", 6), ("[TEXT]", 7), ("[BLOB]", 8)],
)

---

- https://huggingface.co/docs/tokenizers/python/latest/index.html
- https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=pretrainedtokenizer#transformers.PreTrainedTokenizer

In [9]:
input_str = f"{question} [T] {table_id} " + " ".join([f"{column_token} {col} [{typ}]" for col, typ in schema.items()])
input_str

'제 51 기에 삼성전자의 유동자산은 어떻게 돼? [T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [TEXT] [C] bfefrmtrm_dt [TEXT] [C] bfefrmtrm_amount [INTEGER]'

In [192]:
token_re = re.compile(r"\[T\]")

In [290]:
batch_qs = ["제 51 기에 삼성전자의 유동자산은 어떻게 돼?", "2020년도 삼성전자의 유동자산은 얼마?"]
table_str = f"[T]{table_id}" + "".join([f"{column_token}{col}[{typ}]" for col, typ in schema.items()])
batch_ts = [table_str] * len(batch_qs)

In [255]:
new_special_tokens = ["[T]", "[C]", "[INTEGER]", "[REAL]", "[TEXT]", "[BLOB]"]
# new_special_tokens = list(map(lambda x: AddedToken(x, single_word=True, normalized=False), new_special_tokens))
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert', add_special_tokens=True, additional_special_tokens=new_special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [291]:
new_special_tokens_ids = tokenizer.all_special_ids[5:7]
new_special_tokens_ids

[8002, 8003]

In [292]:
encode_input = tokenizer(
    batch_qs, batch_ts, 
    max_length=512, padding=True, truncation=True, return_tensors="pt", 
    return_attention_mask=True, 
    return_special_tokens_mask=False, 
)
encode_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [293]:
question_mask = torch.bitwise_and(encode_input["token_type_ids"] == 0, encode_input["attention_mask"].bool())
table_mask = torch.where(
    (encode_input["input_ids"] == 8002), 
    torch.ones_like(encode_input["input_ids"], dtype=torch.bool), 
    torch.zeros_like(encode_input["input_ids"], dtype=torch.bool)
)
column_mask = torch.where(
    (encode_input["input_ids"] == 8003),
    torch.ones_like(encode_input["input_ids"], dtype=torch.bool), 
    torch.zeros_like(encode_input["input_ids"], dtype=torch.bool)
)

In [294]:
for x in encode_input["input_ids"]:
    print(tokenizer.decode(x))
    print()

[CLS] 제 51 기에 삼성전자의 유동자산은 어떻게 돼?[SEP] [T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [TEXT] [C] bfefrmtrm_dt [TEXT] [C] bfefrmtrm_amount [INTEGER] [SEP]

[CLS] 2020년도 삼성전자의 유동자산은 얼마?[SEP] [T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [TEXT] [C] bfefrmtrm_dt [TEXT] [C] bfefrmtrm_amount [INTEGER] [SEP][PAD][PAD]

In [269]:
for inputs, type_ids in zip(encode_input["input_ids"], encode_input["token_type_ids"]):
    print("--- type_ids = 0")
    print(tokenizer.decode(inputs[type_ids == 0]).replace("[PAD]", ""))
    print("--- type_ids = 1")
    print(tokenizer.decode(inputs[type_ids == 1]))
    print()

--- type_ids = 0
[CLS] 제 51 기에 삼성전자의 유동자산은 어떻게 돼?[SEP]
--- type_ids = 1
[T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [TEXT] [C] bfefrmtrm_dt [TEXT] [C] bfefrmtrm_amount [INTEGER] [SEP]

--- type_ids = 0
[CLS] 작년에 2020년도 삼성전자의 유동자산은 어떻게 돼니?[SEP]
--- type_ids = 1
[T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [

In [295]:
from transformers import BertModel, BertConfig
model = BertModel.from_pretrained("monologg/kobert")
model.resize_token_embeddings(len(tokenizer))

Embedding(8008, 768)

In [296]:
encode_output = model(**encode_input)

In [314]:
question_mask.sum(1)

tensor([16, 14])

In [320]:
question_lengths = question_mask.sum(1).tolist()
batches = torch.split(encode_output.last_hidden_state[question_mask, :], question_lengths)

In [301]:
encode_output.last_hidden_state.size()

torch.Size([2, 247, 768])

In [303]:
encode_output.pooler_output.size()

torch.Size([2, 768])

---

In [111]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt